<a href="https://colab.research.google.com/github/ajignasu/ME592x-Spring2021/blob/main/Assignment3-data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
folderpath = '/content/drive/MyDrive/ColabNotebooks/ME592X/Assignment3/DM/'
# filepath = ''

In [73]:
import numpy as np
from numpy import ndarray
from typing import Tuple
import json, sys, os, torch, torchvision
import pandas as pd
from torch.utils import data
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [74]:

import os
import pandas as pd
import numpy as np


folderpath = '/content/drive/MyDrive/ColabNotebooks/ME592X/Assignment3/DM/'

### Generate a dictionary for folders with four excel files
def input_array1(fName: str):
  
  data_dict = {}
  data_list = []
  temp_list = []
  data_dict[fName + '-0.1.xls'] = []
  data_dict[fName + '-0.01.xls'] = []
  data_dict[fName + '-0.001.xls'] = []
  data_dict[fName + '-0.0001.xls'] = []
  filepath = folderpath + fName + 'C/'
  for file in os.listdir(filepath):
    fileName = file
    fileLocation = filepath + file
    temperature = int(fName)
    temp_list.append(temperature)
    if fileName.endswith('-0.1.xls'):
      input_strainRate = 0.1
    elif fileName.endswith('-0.01.xls'):
      input_strainRate = 0.01
    elif fileName.endswith('-0.001.xls'):
      input_strainRate = 0.001
    else:
      input_strainRate = 0.0001
    temp_list.append(input_strainRate)
    df = pd.read_excel(fileLocation)

    empty_cols = [col for col in df.columns if df[col].isnull().all()]
    df.drop(empty_cols, axis=1, inplace=True)
    df.columns=['Disp', 'Load', 'EStrain', 'EStress', 'TrueStrain', 'TrueStress1', 'PlasticStrain', 'TrueStress2']
    df_data = df.iloc[:,4:6]
    df_update1_data = df_data[df.TrueStress1 != 0]
    df_update2_data = df_update1_data[df.TrueStrain != 0]
    temp_list.append(df_update2_data)
    data_array = np.asarray(temp_list)
  return data_array
     



def input_array2(fName: str):
  
  data_dict = {}
  data_list = []
  temp_list = []
  data_dict[fName + '-0.1.xls'] = []
  data_dict[fName + '-0.01.xls'] = []
  data_dict[fName + '-0.001.xls'] = []
  data_dict[fName + '-0.0001.xls'] = []
  filepath = folderpath + fName + 'C/'
  for file in os.listdir(filepath):
    fileName = file
    fileLocation = filepath + file
    temperature = int(fName)
    temp_list.append(temperature)
    if fileName.endswith('-0.1.xls'):
      input_strainRate = 0.1
    elif fileName.endswith('-0.01.xls'):
      input_strainRate = 0.01
    elif fileName.endswith('-0.001.xls'):
      input_strainRate = 0.001
    else:
      input_strainRate = 0.0001
    temp_list.append(input_strainRate)
    df = pd.read_excel(fileLocation)

    empty_cols = [col for col in df.columns if df[col].isnull().all()]
    df.drop(empty_cols, axis=1, inplace=True)
    if len(df.columns) == 8:
      df.columns=['Disp', 'Load', 'EStrain', 'EStress', 'TrueStrain', 'TrueStress1', 'PlasticStrain', 'TrueStress2']
    else:
      df.columns=['Disp', 'Load', 'EStrain', 'EStress', 'TrueStrain', 'TrueStress1']
    
    df_data = df.iloc[:,4:6]
    df_update1_data = df_data[df.TrueStress1 != 0]
    df_update2_data = df_update1_data[df.TrueStrain != 0]
    temp_list.append(df_update2_data)
    data_array = np.asarray(temp_list)
  return data_array  




In [75]:
# 50C, 100C, 150C, 200C, 250C, 300C, 350C  - each has four excel files

temp_50C = input_array1('50')
temp_100C = input_array1('100')
temp_150C = input_array1('150')
temp_200C = input_array1('200')
temp_250C = input_array1('250')
temp_300C = input_array1('300')
temp_350C = input_array1('350')


# 450C, 500C, 550C, 600C, 650C -  each has three excel files

temp_450C = input_array2('450')
temp_500C = input_array2('500')
temp_550C = input_array2('550')
temp_600C = input_array2('600')
temp_650C = input_array2('650')

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is de

In [ ]:
#### Naive learning

import os
import torch
import torch.nn as nn
import torch.nn.functional as F


class Fully_Connected(nn.Module):
    def __init__(self, in_dims, out_dims):
        super(Fully_Connected, self).__init__()
        
        self.layer_1 = nn.Linear(in_dims, 8*out_dims)
        self.layer_2 = nn.Linear(8*out_dims, 4*out_dims)
        self.layer_3 = nn.Linear(4*out_dims, 2*out_dims)
        self.layer_4 = nn.Linear(2*out_dims, out_dims)

    def forward(self, x):
        x = F.relu(self.layer_1(x))
        x = F.relu(self.layer_2(x))
        x = F.relu(self.layer_3(x))
        return self.layer_4(x)


# Recurrent neural network (many-to-one)
# https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/02-intermediate/recurrent_neural_network/main.py#L39-L58
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial hidden and cell states 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
#####  TESTING CODE - DELETE AFTER GENERATING MAIN FUNCTIONS #####

# # cleanup input excel files

# # for file in os.listdir(folderpath+'/50C'):
# #   # for file in os.listdir(folderpath + '/50C'):
# #     if file.endswith('xls'):
# #       print(file)
# import os
# import pandas as pd

# folderpath = '/content/drive/MyDrive/ColabNotebooks/ME592X/Assignment3/DM/'

# # for file in os.listdir(folderpath + '/50C'):
#   # count = 1
# filepath = folderpath + '/50C/50-0.1.xls'
#   # for file in folder + '/50C':
# df = pd.read_excel(filepath)

# # remove empty columns
# empty_cols = [col for col in df.columns if df[col].isnull().all()]
# df.drop(empty_cols, axis=1, inplace=True)
# # print(df)

# #remove whitespace from columns names
# df.columns=['Disp', 'Load', 'EStrain', 'EStress', 'TrueStrain', 'TrueStress1', 'PlasticStrain', 'TrueStress2']
# # print(df)

# df = df[df.PlasticStrain != 0]
# print(df)
#   # if count == 1:
#   #   exit()



          Disp  Load   EStrain  ...  TrueStress1  PlasticStrain  TrueStress2
20    0.120000  0.60  0.004000  ...   100.400004       0.000095   100.400004
27    0.190000  0.94  0.006333  ...   157.658888       0.000194   157.658888
32    0.260000  1.29  0.008667  ...   216.863327       0.000212   216.863327
37    0.310000  1.57  0.010333  ...   264.370564       0.000019   264.370564
38    0.320000  1.61  0.010667  ...   271.195558       0.000084   271.195558
..         ...   ...       ...  ...          ...            ...          ...
216  16.709999  3.81  0.557000  ...   988.694966       0.404387   988.694966
217  16.950001  3.80  0.565000  ...   991.166670       0.409416   991.166670
218  17.410000  3.79  0.580333  ...   998.243876       0.418892   998.243876
219  17.580000  3.69  0.586000  ...   975.390014       0.423358   975.390014
220  17.830000  2.90  0.594333  ...   770.594469       0.436547   770.594469

[187 rows x 8 columns]


In [ ]:
# import os
# import pandas as pd

folderpath = '/content/drive/MyDrive/ColabNotebooks/ME592X/Assignment3/DM/'


In [10]:
### generate dictionary for input data

import os
import pandas as pd

folderpath = '/content/drive/MyDrive/ColabNotebooks/ME592X/Assignment3/DM/'

# ### Input dictionary for folders with four excel files
# def input_dict1(fName: str):
#   '''  Generate a dictionary for every input temperature '''
#   data_dict = {}
#   data_dict[fName + '-0.1.xls'] = []
#   data_dict[fName + '-0.01.xls'] = []
#   data_dict[fName + '-0.001.xls'] = []
#   data_dict[fName + '-0.0001.xls'] = []
#   filepath = folderpath + fName + 'C/'
#   for file in os.listdir(filepath):
#     fileName = file
#     fileLocation = filepath + file

#     df = pd.read_excel(fileLocation)
#     # df.columns=['Disp', 'Load', 'EStrain', 'EStress', 'TrueStrain', 'TrueStress1', 'PlasticStrain', 'TrueStress2']
#     # df_new = df['True Strain', 'TrueStress1']
#     # print(df_new)
#     empty_cols = [col for col in df.columns if df[col].isnull().all()]
#     df.drop(empty_cols, axis=1, inplace=True)
#     df.columns=['Disp', 'Load', 'EStrain', 'EStress', 'TrueStrain', 'TrueStress1', 'PlasticStrain', 'TrueStress2']
#     # df2 = df.iloc[:,4:6] ### This should be for True Stress and Plastic Strain; not True Strain
#     df2 = df.iloc[:,5:7] ## Try this out
#     df_update_TrueStress1 = df2[df2.TrueStress1 != 0]
#     df_new = df_update_TrueStress1[df_update_TrueStress1.PlasticStrain != 0]
#     # print(fileName)
#     # print(df_new)

#     if fileName.endswith('-0.1.xls'):
#       data_dict[fName + '-0.1.xls'].append(df_new)
#     elif fileName.endswith('-0.01.xls'):
#       data_dict[fName + '-0.01.xls'].append(df_new)
#     elif fileName.endswith('-0.001.xls'):
#       data_dict[fName + '-0.001.xls'].append(df_new)
#     else:
#       data_dict[fName + '-0.0001.xls'].append(df_new)
#   return data_dict
      

# temp_100C = input_dict1('100')
# print(temp_100C)

# ### Input dictionary for folders with three excel files
# def input_dict2(fName: str):
#   '''  Generate a dictionary for every input temperature '''
#   data_dict = {}
#   # data_dict[fName + '-0.1.xls'] = []
#   data_dict[fName + '-0.01.xls'] = []
#   data_dict[fName + '-0.001.xls'] = []
#   data_dict[fName + '-0.0001.xls'] = []
#   filepath = folderpath + fName + 'C/'
#   for file in os.listdir(filepath):
#     fileName = file
#     fileLocation = filepath + file

#     df = pd.read_excel(fileLocation)
#     empty_cols = [col for col in df.columns if df[col].isnull().all()]
#     df.drop(empty_cols, axis=1, inplace=True)
#     if len(df.columns) == 8:
#       df.columns=['Disp', 'Load', 'EStrain', 'EStress', 'TrueStrain', 'TrueStress1', 'PlasticStrain', 'TrueStress2']
#     else:
#       df.columns=['Disp', 'Load', 'EStrain', 'EStress', 'TrueStrain', 'TrueStress1']
#     df2 = df.iloc[:,4:6]
#     df_update_TrueStrain = df2[df2.TrueStrain != 0]
#     df_new = df_update_TrueStrain[df_update_TrueStrain.TrueStress1 != 0]
    
#     if fileName.endswith('-0.01.xls'):
#       data_dict[fName + '-0.01.xls'].append(df_new)
#     elif fileName.endswith('-0.001.xls'):
#       data_dict[fName + '-0.001.xls'].append(df_new)
#     else:
#       data_dict[fName + '-0.0001.xls'].append(df_new)
#   return data_dict


{'100-0.1.xls': [     TrueStress1  PlasticStrain
34    242.560010       0.000085
36    259.575549       0.000006
37    264.719453       0.000113
38    269.866671       0.000219
39    278.483329       0.000503
..           ...            ...
210   853.825589       0.378807
211   839.452756       0.383381
212   813.688909       0.387124
213   797.454468       0.389793
214   761.805009       0.398077

[180 rows x 2 columns]], '100-0.01.xls': [      TrueStress1  PlasticStrain
163    273.240001       0.000072
166    278.391663       0.000178
167    280.078883       0.000105
168    281.858882       0.000357
169    283.546658       0.000284
...           ...            ...
1447   896.715531       0.455990
1448   869.906681       0.457762
1449   832.284446       0.460207
1450   805.396674       0.461982
1451   548.555565       0.474343

[1287 rows x 2 columns]], '100-0.001.xls': [      TrueStress1  PlasticStrain
25      45.090002       0.000041
31      50.116669       0.000156
34      53.45777

In [12]:
# ### TESTING FOR GENERATING INPUT LIST

# import os
# import pandas as pd
# import numpy as np


# folderpath = '/content/drive/MyDrive/ColabNotebooks/ME592X/Assignment3/DM/'

# ### Generate a dictionary for folders with four excel files
# def input_dict1(fName: str):
#   '''  Generate a dictionary for every input temperature '''
#   data_list = []
#   temp_list = []
#   true_stress = []
#   plastic_strain = []
#   # data_dict[fName + '-0.1.xls'] = []
#   # data_dict[fName + '-0.01.xls'] = []
#   # data_dict[fName + '-0.001.xls'] = []
#   # data_dict[fName + '-0.0001.xls'] = []
#   filepath = folderpath + fName + 'C/'
#   for file in os.listdir(filepath):
#     fileName = file
#     fileLocation = filepath + file
#     temperature = int(fName)
#     temp_list.append(temperature)
#     if fileName.endswith('-0.1.xls'):
#       input_strainRate = 0.1
#     elif fileName.endswith('-0.01.xls'):
#       input_strainRate = 0.01
#     elif fileName.endswith('-0.001.xls'):
#       input_strainRate = 0.001
#     else:
#       input_strainRate = 0.0001
#     temp_list.append(input_strainRate)
#     df = pd.read_excel(fileLocation)
#     # df.columns=['Disp', 'Load', 'EStrain', 'EStress', 'TrueStrain', 'TrueStress1', 'PlasticStrain', 'TrueStress2']
#     # df_new = df['True Strain', 'TrueStress1']
#     # print(df_new)
#     empty_cols = [col for col in df.columns if df[col].isnull().all()]
#     df.drop(empty_cols, axis=1, inplace=True)
#     df.columns=['Disp', 'Load', 'EStrain', 'EStress', 'TrueStrain', 'TrueStress1', 'PlasticStrain', 'TrueStress2']
#     df_TrueStress1 = df.iloc[:,5:6]
#     df_update_TrueStress1 = df_TrueStress1[df.TrueStress1 != 0]
#     # print(df_update_TrueStress1)
#     # print(type(df_update_TrueStress1))
#     # print(df_update_TrueStress1.size)
#     shape1 = df_update_TrueStress1.shape
#     # print(df_update_TrueStress1.shape)
#     # print(df_update_TrueStress1.ndim)
#     # df_TrueStress1_list = df_update_TrueStress1.values.tolist()
#     TrueStress1_array = df_update_TrueStress1.to_numpy()
#     # print('array shape')
#     # print(TrueStress1_array.shape)
#     # df_TrueStress1_reshape = np.reshape(df_TrueStress1_array, shape1)
#     true_stress.append(TrueStress1_array)
#     temp_list.append(true_stress)
#     # print('temp list for true stress')
#     # print(temp_list)
#     # print('true stress')
#     # print(df_TrueStress1_reshape.shape)
#     # print(np.shape(true_stress))
#     # print(df_TrueStress1_list)
    
#     df_PlasticStrain = df.iloc[:,6:7]
#     # print(df_PlasticStrain)
#     df_update_PlasticStrain = df_PlasticStrain[df.PlasticStrain != 0]
#     print(df_update_PlasticStrain)
#     shape2 = df_update_PlasticStrain.shape
#     PlasticStrain_array = df_update_PlasticStrain.to_numpy()
#     # df_PlasticStrain_reshape = np.reshape(df_PlasticStrain_list, shape2)
#     plastic_strain.append(PlasticStrain_array)
#     temp_list.append(plastic_strain)
#     # print('temp list for plastic strain')
#     # print(temp_list)
#     # print('plastic strain')
#     # print(df_PlasticStrain_reshape.shape)
#     # df_new = df_update_TrueStress1[df_update_TrueStress1.PlasticStrain != 0]
#     # df_new_list = df_new.values.tolist()
#     # print(df_new_list)
#     # print(type(df_new))
#     # temp_list.append(df_new)
#     # print(temp_list)
#     # data_list.append(temp_list)
#     # print(data_list)
#     # temp_list.clear()
#     # print(temp_list)
    
#     # print(fileName)
#     # print(df_new)

#     # if fileName.endswith('-0.1.xls'):
#     #   data_dict[fName + '-0.1.xls'].append(df_new)
#     # elif fileName.endswith('-0.01.xls'):
#     #   data_dict[fName + '-0.01.xls'].append(df_new)
#     # elif fileName.endswith('-0.001.xls'):
#     #   data_dict[fName + '-0.001.xls'].append(df_new)
#     # else:
#     #   data_dict[fName + '-0.0001.xls'].append(df_new)
#   # return data_dict
      

# temp_100C = input_dict1('100')
# print(temp_100C)

      PlasticStrain
163        0.000072
166        0.000178
167        0.000105
168        0.000357
169        0.000284
...             ...
1447       0.455990
1448       0.457762
1449       0.460207
1450       0.461982
1451       0.474343

[1287 rows x 1 columns]
      PlasticStrain
25         0.000041
31         0.000156
34         0.000011
40         0.000125
41         0.000125
...             ...
2218       0.524956
2219       0.525464
2220       0.525774
2221       0.526028
2222       0.526536

[2181 rows x 1 columns]
     PlasticStrain
34        0.000085
36        0.000006
37        0.000113
38        0.000219
39        0.000503
..             ...
210       0.378807
211       0.383381
212       0.387124
213       0.389793
214       0.398077

[180 rows x 1 columns]
      PlasticStrain
184        0.000015
189        0.000141
190        0.000042
192        0.000266
193        0.000167
...             ...
4800       0.422871
4801       0.423025
4802       0.423334
4803       0.42348

In [71]:
#### Remove before submission


# import os
# import pandas as pd
# import numpy as np


# folderpath = '/content/drive/MyDrive/ColabNotebooks/ME592X/Assignment3/DM/'

# def input_array2(fName: str):
  
#   data_dict = {}
#   data_list = []
#   temp_list = []
#   data_dict[fName + '-0.1.xls'] = []
#   data_dict[fName + '-0.01.xls'] = []
#   data_dict[fName + '-0.001.xls'] = []
#   data_dict[fName + '-0.0001.xls'] = []
#   filepath = folderpath + fName + 'C/'
#   for file in os.listdir(filepath):
#     fileName = file
#     fileLocation = filepath + file
#     temperature = int(fName)
#     temp_list.append(temperature)
#     if fileName.endswith('-0.1.xls'):
#       input_strainRate = 0.1
#     elif fileName.endswith('-0.01.xls'):
#       input_strainRate = 0.01
#     elif fileName.endswith('-0.001.xls'):
#       input_strainRate = 0.001
#     else:
#       input_strainRate = 0.0001
#     temp_list.append(input_strainRate)
#     df = pd.read_excel(fileLocation)

#     empty_cols = [col for col in df.columns if df[col].isnull().all()]
#     df.drop(empty_cols, axis=1, inplace=True)
#     if len(df.columns) == 8:
#       df.columns=['Disp', 'Load', 'EStrain', 'EStress', 'TrueStrain', 'TrueStress1', 'PlasticStrain', 'TrueStress2']
#     else:
#       df.columns=['Disp', 'Load', 'EStrain', 'EStress', 'TrueStrain', 'TrueStress1']
    
#     df_data = df.iloc[:,4:6]
#     df_update1_data = df_data[df.TrueStress1 != 0]
#     df_update2_data = df_update1_data[df.TrueStrain != 0]
#     temp_list.append(df_update2_data)
#     data_array = np.asarray(temp_list)
#   return data_array






temp_450C = input_array2('450')
temp_500C = input_array2('500')
temp_550C = input_array2('550')
print(temp_550C)
temp_600C = input_array2('600')
temp_650C = input_array2('650')





/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  retu

[550 0.001
       TrueStrain  TrueStress1
319     0.000333     3.473380
320     0.000666     3.474537
321     0.000999     3.475694
322     0.001332     3.476852
323     0.001665     3.478009
...          ...          ...
2322    0.425486   308.192127
2323    0.425486   305.535301
2324    0.425703   297.629626
2325    0.425703   281.685165
2326    0.425703   241.824077

[2008 rows x 2 columns]
 550 0.0001
        TrueStrain  TrueStress1
12       0.003992     1.743056
16       0.000666     1.737268
17       0.001000     1.737847
19       0.001000     1.737847
21       0.001000     1.737847
...           ...          ...
12939    0.499158   208.776626
12940    0.499158   211.636577
12941    0.499158   214.496528
12942    0.499158   211.636577
12943    0.499158   208.776626

[12754 rows x 2 columns]
 550 0.01
       TrueStrain  TrueStress1
1       0.008960    21.020833
2       0.000333    34.733797
3       0.000666    38.219907
4       0.001000    41.708332
5       0.001000    48.659722
.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  retu

In [51]:
### testing Def input_dict2


# def input_dict2(fName: str):
#   '''  Generate a dictionary for every input temperature '''
#   data_dict = {}
#   # data_dict[fName + '-0.1.xls'] = []
#   data_dict[fName + '-0.01.xls'] = []
#   data_dict[fName + '-0.001.xls'] = []
#   data_dict[fName + '-0.0001.xls'] = []
#   filepath = folderpath + fName + 'C/'
#   for file in os.listdir(filepath):
#     fileName = file
#     fileLocation = filepath + file

#     df = pd.read_excel(fileLocation)
#     # df.columns=['Disp', 'Load', 'EStrain', 'EStress', 'TrueStrain', 'TrueStress1', 'PlasticStrain', 'TrueStress2']
#     # df_new = df['True Strain', 'TrueStress1']
#     # print(df_new)
#     empty_cols = [col for col in df.columns if df[col].isnull().all()]
#     df.drop(empty_cols, axis=1, inplace=True)
#     if len(df.columns) == 8:
#       df.columns=['Disp', 'Load', 'EStrain', 'EStress', 'TrueStrain', 'TrueStress1', 'PlasticStrain', 'TrueStress2']
#     else:
#       df.columns=['Disp', 'Load', 'EStrain', 'EStress', 'TrueStrain', 'TrueStress1']
#     df2 = df.iloc[:,4:6]
#     df_update_TrueStrain = df2[df2.TrueStrain != 0]
#     df_new = df_update_TrueStrain[df_update_TrueStrain.TrueStress1 != 0]
#     # print(fileName)
#     # print(df_new)

#     # if fileName.endswith('-0.1.xls'):
#       # data_dict[fName + '-0.1.xls'].append(df_new)
#       # print(fileName)
#       # print(data_dict[fName + '-0.1.xls'])
#     if fileName.endswith('-0.01.xls'):
#       data_dict[fName + '-0.01.xls'].append(df_new)
#     elif fileName.endswith('-0.001.xls'):
#       data_dict[fName + '-0.001.xls'].append(df_new)
#     else:
#       data_dict[fName + '-0.0001.xls'].append(df_new)
#   return data_dict

# temp_550C = input_dict2('550')
# print('printing 550C data')
# print(temp_550C)
# temp_600C = input_dict2('600')
# print('printing 600C data')
# print(temp_600C)
# temp_650C = input_dict2('650')
# print('printing 650C data')
# print(temp_650C)


printing 550C data
{'550-0.01.xls': [      TrueStrain  TrueStress1
1       0.008960    21.020833
2       0.000333    34.733797
3       0.000666    38.219907
4       0.001000    41.708332
5       0.001000    48.659722
...          ...          ...
2388    0.352767   303.869793
2389    0.352767   298.928826
2390    0.353002   296.527788
2391    0.353002   286.643509
2392    0.353236   261.994199

[2392 rows x 2 columns]], '550-0.001.xls': [      TrueStrain  TrueStress1
319     0.000333     3.473380
320     0.000666     3.474537
321     0.000999     3.475694
322     0.001332     3.476852
323     0.001665     3.478009
...          ...          ...
2322    0.425486   308.192127
2323    0.425486   305.535301
2324    0.425703   297.629626
2325    0.425703   281.685165
2326    0.425703   241.824077

[2008 rows x 2 columns]], '550-0.0001.xls': [       TrueStrain  TrueStress1
12       0.003992     1.743056
16       0.000666     1.737268
17       0.001000     1.737847
19       0.001000     1.7378

In [ ]:
# class TopoDataset1(data.Dataset):
#     'PyTorch dataset for Topology Optimization'
    
#     def __init__(self, data_path, model_type='regressor', mode='train'):
#         'Initialization'
#         ### Identify unique model dataset path
#         if model_type == 'regressor':
#             self.model_type = os.path.join(data_path,'regressor_data')
#         elif model_type == 'generator':
#             self.model_type = os.path.join(data_path,'generator_data')
#         ### Identify dataset subset
#         if mode == 'train':
#             self.data_path = os.path.join(self.model_type,'train')
#         elif mode == 'test':
#             self.data_path = os.path.join(self.model_type,'test')
#         elif mode == 'validation':
#             self.data_path = os.path.join(self.model_type,'validation')

#         self.initial_SE = np.load(os.path.join(self.data_path, 'initial_SE.npz'))['arr_0']
#         self.initial_D = np.load(os.path.join(self.data_path, 'initial_D.npz'))['arr_0']
#         self.final_SE = np.load(os.path.join(self.data_path, 'final_SE.npz'))['arr_0']
#         self.final_D = np.load(os.path.join(self.data_path, 'final_D.npz'))['arr_0']      
#     def __len__(self):
#         'Denotes the total number of samples'
#         return self.initial_SE.shape[0]
#     def __getitem__(self, index):
#         'Generates one sample of data'
#         data_dict = {}
#         data_dict['initial_SE'] = torch.FloatTensor(self.initial_SE[index])
#         data_dict['initial_D'] = torch.FloatTensor(self.initial_D[index])
#         data_dict['final_SE'] = torch.FloatTensor(self.final_SE[index])
#         data_dict['final_D'] = torch.FloatTensor(self.final_D[index])
#         return data_dict